In [1]:
require 'nn'
require 'hdf5'

### LOADING THE DATA AND CONVERTING IT TO LOGREG FW:

In [2]:
myFile = hdf5.open('PTB.hdf5','r')

In [3]:
data = myFile:all()
myFile:close()

In [ ]:
data

In [4]:
train_input_word_windows = data['train_input_word_windows']
train_output = data['train_output']

In [5]:
train = train_input_word_windows:clone()

In [6]:
for i = 1, train:size(1) do
    for j = 1, 5 do
        train[i][j] = (j-1)*100002 + train[i][j]
    end
end

In [56]:
train_input_cap_windows = data['train_input_cap_windows']
train_cap = train_input_cap_windows:clone()
for i = 1, train:size(1) do
    for j = 1, 5 do
        train_cap[i][j] = (j-1)*4 + train_cap[i][j]
    end
end

### TRAINING:

In [50]:
linreg = nn.Sequential()

In [78]:
par = nn.ParallelTable()

In [79]:
par:add(nn.LookupTable(5*data['nwords'][1],data['nclasses'][1])) -- first child
par:add(nn.LookupTable(5*4,data['nclasses'][1])) 

In [83]:
linreg_wc = nn.Sequential()
linreg_wc:add(par)
linreg_wc:add(nn.CAddTable())
linreg_wc:add(nn.Sum(2))
linreg_wc:add(nn.Add(45))
linreg_wc:add(nn.LogSoftMax())

In [75]:
-- Sanity check:
linreg_wc:forward({train:narrow(1,5, 10),train_cap:narrow(1,5, 10)})

Columns 1 to 8
 -0.8186 -11.8379  -5.6438 -14.4388  -7.3078  -8.3703  -8.0455  -0.8975
 -5.5997 -14.0458  -3.1820  -7.5692  -0.7484 -11.1423 -13.4242  -9.8909
 -5.4569 -15.7438  -5.2446 -10.3546  -3.5530  -5.3203  -9.6729  -9.7699
 -1.4952  -7.5692  -5.1844 -15.0385  -2.8423  -4.4383 -13.0080  -6.8295
 -1.9215 -10.5381  -6.2333 -12.1216  -3.6143  -7.8784 -11.1044  -4.7236
 -0.5884 -13.6767  -5.3946 -14.6605  -1.9824 -10.0362 -11.6843  -4.7942
 -4.2065 -14.5070  -5.2114 -14.0529  -3.9096  -6.7314  -9.4918  -9.6309
 -2.8374  -7.4668  -4.2657  -9.7529  -3.2895  -0.8971  -9.6997  -6.4985
 -4.1349 -13.2034  -1.0139 -10.0785  -5.6546  -4.6099 -12.5774  -4.5594
 -0.0757  -6.4610  -6.0134 -10.3878  -5.1925  -7.4950 -10.9558 -11.9017

Columns 9 to 16
 -9.6030 -12.4806  -8.4809  -8.5890  -5.7233  -3.5286 -13.4280  -9.3672
 -4.9246  -4.1400  -4.8397  -4.9063  -3.4000  -6.1977  -6.7050  -3.9693
 -9.6097  -7.9021  -5.1130  -8.8636  -4.2414  -6.1091 -13.8390 -12.2487
 -7.4427  -7.8708  -4.2637 -10.4

946  -9.0741
 -9.0156  -8.2138 -12.1546  -9.6799  -5.0514
[torch.DoubleTensor of size 10x45]



In [ ]:
-- linreg_w:add(nn.LookupTable(5*data['nwords'][1],data['nclasses'][1]))
-- linreg_w:add(nn.Sum(2))
-- linreg_w:add(nn.Add(data['nclasses'][1]))
-- linreg_w:add(nn.LogSoftMax())

In [84]:
criterion = nn.ClassNLLCriterion()

In [87]:
train_output:size(1)

912666	


In [ ]:
eta = 0.01
input_w = torch.Tensor(100,5)
input_c = torch.Tensor(100,5)
output = torch.Tensor(100)

for i = 1,1 do
   
    --28520
    for j = 1,9126 do
        linreg_wc:zeroGradParameters()
        
        input_w = train:narrow(1, (j-1)*100+1, 100)
        input_c = train_cap:narrow(1, (j-1)*100+1, 100)
        preds = linreg_wc:forward({input_w,input_c})
        
        output = train_output:narrow(1,(j-1)*100+1, 100)
        
        loss = criterion:forward(preds, output)
        
        if j % 500 == 0 then
            print(loss)
        end
        
        dLdpreds = criterion:backward(preds, output)
        
        linreg_wc:backward({input_w,input_c}, dLdpreds)
        
        linreg:updateParameters(eta)
        
    end
    
end


### VALIDATION:

In [ ]:
val = data['valid_input_word_windows']:clone()
for i = 1, val:size(1) do
    for j = 1, 5 do
        val[i][j] = (j-1)*100002 + val[i][j]
    end
end

In [ ]:
pred_val = linreg:forward(val)

In [ ]:
m,a = pred_val:max(2)

In [ ]:
acc = 0
for i = 1, data['valid_output']:size(1) do
    if a[i] == data['valid_output'][i] then
        acc = acc + 1
    end
end
print(acc)

In [ ]:
minibatch =  torch.range((j-1)*32+1, j*32):type('torch.LongTensor')

In [ ]:
preds = linreg:forward(train:index(1,minibatch))

In [ ]:
loss = mse:forward(preds, train_output:index(1,minibatch))
        

In [ ]:
dLdpreds = mse:backward(preds, train_output:index(1,minibatch))

In [ ]:
linreg:backward(X, dLdpreds)

In [ ]:
x = torch.LongTensor({1,2,5})